Problem Description:
Suppose we have a delivery company with a fleet of three vehicles tasked with delivering packages to six customers from a central depot. Each customer has a specific demand representing the number of packages to be delivered, and there are known distances between each pair of customers and the depot.

Solution:
We can use the Google OR-Tools library in Python to solve this VRP instance. Here's how we can implement and solve the problem:

In [3]:
#Imports:
#The code imports necessary modules from Google OR-Tools, namely routing_enums_pb2 and pywrapcp.

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

#Data Model Creation Function (create_data_model):
#This function creates a data model for the VRP.
#It initializes a dictionary called data with information about the distance matrix (distances between locations), demands (package demands at each location), number of vehicles, vehicle capacity, and the depot location (where vehicles start and end their routes).
#The data model is based on a sample problem with predefined distance matrix, demands, and other parameters.

def create_data_model(num_vehicles):
    data = {}
    data['distance_matrix'] = [
        [0, 10, 20, 30, 40, 50],  # Distance from depot (0) to each customer
        [10, 0, 15, 25, 35, 45],  # Distance between customers
        [20, 15, 0, 10, 20, 30],
        [30, 25, 10, 0, 10, 20],
        [40, 35, 20, 10, 0, 10],
        [50, 45, 30, 20, 10, 0],
    ]
    data['demands'] = [0, 5, 8, 10, 6, 7]  # Demand for each customer
    data['num_vehicles'] = num_vehicles  # Number of vehicles
    data['vehicle_capacity'] = 15  # Capacity of each vehicle
    data['depot'] = 0  # Index of the depot
    return data

#Printing Solution Function (print_solution):
#This function prints the solution of the VRP.
#It takes the data model, routing manager, routing model, and solution as inputs.
#It iterates over each vehicle's route and prints the sequence of customers visited, the distance traveled, and the load (demand) of each route.
#Finally, it prints the total distance and total load of all routes.

def print_solution(data, manager, routing, solution):
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            next_node_index = manager.IndexToNode(solution.Value(routing.NextVar(index)))
            route_distance += data['distance_matrix'][node_index][next_node_index]
            route_load += data['demands'][node_index]
            plan_output += f'  Customer {node_index} -> '
            index = solution.Value(routing.NextVar(index))
        node_index = manager.IndexToNode(index)
        total_distance += route_distance
        total_load += route_load
        plan_output += f'Customer {node_index}\n'
        plan_output += f'Distance of the route: {route_distance}m\n'
        plan_output += f'Load of the route: {route_load}\n'
        print(plan_output)
    print(f'Total distance of all routes: {total_distance}m')
    print(f'Total load of all routes: {total_load}')


#VRP Solver Function (solve_vrp):
#This function solves the VRP.
#It first initializes the data model with a single vehicle to calculate the minimum number of vehicles required (calculate_min_vehicles).
#Then, it iterates over a range of possible numbers of vehicles, starting from the minimum required and increasing gradually.
#For each number of vehicles, it creates a data model with that number of vehicles, sets up the routing model, solves the VRP, and prints the solution.
#It breaks out of the loop once a solution is found.    

def solve_vrp():
    data = create_data_model(1)  # Initialize 'data' with a default value
    min_vehicles = calculate_min_vehicles(data)  # Pass 'data' to calculate_min_vehicles
    for num_vehicles in range(min_vehicles, min_vehicles + 5):  # Set a maximum limit for number of vehicles
        data['num_vehicles'] = num_vehicles  # Update the number of vehicles in 'data'
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
        routing = pywrapcp.RoutingModel(manager)
 
# demand_callback Function:
#This function is responsible for providing the demand (package demand) at each location (customer).
#It takes a single argument from_index, which represents the index of the current location (customer) in the routing model.
#Inside the function, it converts the from_index to the actual node index using the IndexToNode method of the routing manager (manager).
#Then, it retrieves the demand of the corresponding node from the data dictionary using the obtained node index.
#Finally, it returns the demand of the current location.


        def demand_callback(from_index):
            from_node = manager.IndexToNode(from_index)
            return data['demands'][from_node]

        demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
        routing.AddDimensionWithVehicleCapacity(
            demand_callback_index,
            0,  # null capacity slack
            [data['vehicle_capacity']] * data['num_vehicles'],  # vehicle maximum capacities
            True,  # start cumul to zero
            'Capacity'
        )
        

#distance_callback Function:
#This function is responsible for providing the distance between two locations (customers).
#It takes two arguments, from_index and to_index, which represent the indices of the current and next locations (customers) in the routing model, respectively.
#Similar to the demand_callback function, it converts the indices to actual node indices using the IndexToNode method of the routing manager (manager).
#Then, it retrieves the distance between the corresponding nodes from the data dictionary using the obtained node indices.
#Finally, it returns the distance between the current and next locations.

        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
        )

        solution = routing.SolveWithParameters(search_parameters)
        if solution:
            print(f"Solution found with {num_vehicles} vehicles:")
            print_solution(data, manager, routing, solution)
            break
    else:
        print("Solution not found within the given vehicle range.")

#Function to Calculate Minimum Number of Vehicles (calculate_min_vehicles):
#This function calculates the minimum number of vehicles required to fulfill all demands.
#It sums up the demands of all customers, divides by the vehicle capacity, and rounds up to the nearest integer.        

def calculate_min_vehicles(data):  # Pass 'data' as argument
    total_demand = sum(data['demands'])
    min_vehicles = (total_demand + data['vehicle_capacity'] - 1) // data['vehicle_capacity']  # Round up
    return min_vehicles

solve_vrp()


Solution found with 3 vehicles:
Route for vehicle 0:
  Customer 0 ->   Customer 2 ->   Customer 1 -> Customer 0
Distance of the route: 45m
Load of the route: 13

Route for vehicle 1:
  Customer 0 ->   Customer 3 -> Customer 0
Distance of the route: 60m
Load of the route: 10

Route for vehicle 2:
  Customer 0 ->   Customer 4 ->   Customer 5 -> Customer 0
Distance of the route: 100m
Load of the route: 13

Total distance of all routes: 205m
Total load of all routes: 36


In [4]:
import numpy as np

# Define the distance matrix (distance between each pair of customers)
distance_matrix = np.array([
    [0, 10, 20, 30, 40, 50],  
    [10, 0, 15, 25, 35, 45],  
    [20, 15, 0, 10, 20, 30],
    [30, 25, 10, 0, 10, 20],
    [40, 35, 20, 10, 0, 10],
    [50, 45, 30, 20, 10, 0],
])

# Define demands (package demands at each customer)
demands = [0, 5, 8, 10, 6, 7]

# Define vehicle capacity
vehicle_capacity = 15

# Define depot (starting point)
depot = 0

def calculate_savings(distance_matrix):
    n = len(distance_matrix)
    savings = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            savings[i, j] = distance_matrix[i, depot] + distance_matrix[j, depot] - distance_matrix[i, j]
    return savings

def clarke_wright_savings(distance_matrix, demands, vehicle_capacity):
    n = len(distance_matrix)
    savings = calculate_savings(distance_matrix)
    routes = [[depot] for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            routes[i].append(j)
            routes[j].append(i)
    
    merged_routes = []

    # Sort the savings in descending order
    sorted_indices = np.dstack(np.unravel_index(np.argsort(-savings.ravel()), (n, n))).reshape(-1, 2)
    
    for pair in sorted_indices:
        i, j = pair
        # Check if adding the pair will violate capacity constraints
        if len(merged_routes) < 1:
            route1_capacity = sum(demands[node] for node in routes[i])
            route2_capacity = sum(demands[node] for node in routes[j])
            if route1_capacity + route2_capacity <= vehicle_capacity:
                merged_routes.append(routes[i] + routes[j])
                routes[i] = []
                routes[j] = []
    
    # Remove empty routes
    merged_routes = [route for route in merged_routes if route]
    return merged_routes

# Run Clarke-Wright Savings Algorithm
merged_routes = clarke_wright_savings(distance_matrix, demands, vehicle_capacity)

# Print the merged routes
for i, route in enumerate(merged_routes):
    print(f"Route {i+1}: {route}")


In [5]:
import numpy as np

def calculate_distance_matrix(locations):
    n = len(locations)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            distance_matrix[i, j] = distance_matrix[j, i] = np.linalg.norm(locations[i] - locations[j])
    return distance_matrix

def calculate_savings(distance_matrix):
    n = len(distance_matrix)
    savings = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            savings[i, j] = distance_matrix[i, 0] + distance_matrix[j, 0] - distance_matrix[i, j]
    return savings

def clarke_wright_savings(distance_matrix, vehicle_capacity):
    n = len(distance_matrix)
    savings = calculate_savings(distance_matrix)
    sorted_indices = np.dstack(np.unravel_index(np.argsort(-savings.ravel()), (n, n))).reshape(-1, 2)
    
    routes = [[i] for i in range(1, n)]
    merged_routes = []

    for pair in sorted_indices:
        i, j = pair
        route1 = routes[i-1]
        route2 = routes[j-1]
        if len(merged_routes) < 1:
            route1_demand = sum(demands[node] for node in route1)
            route2_demand = sum(demands[node] for node in route2)
            if route1_demand + route2_demand <= vehicle_capacity:
                merged_routes.append(route1 + route2[::-1])
                routes[i-1] = []
                routes[j-1] = []

    merged_routes = [route for route in merged_routes if route]
    return merged_routes

# Define locations of customers (excluding the depot)
locations = np.array([
    [0, 0],  # Depot
    [1, 2],
    [3, 4],
    [5, 6],
    [7, 8],
    [9, 10]
])

# Define demands (package demands at each customer)
demands = [0, 5, 8, 10, 6, 7]

# Define vehicle capacity
vehicle_capacity = 15

# Calculate distance matrix
distance_matrix = calculate_distance_matrix(locations)

# Run Clarke-Wright Savings Algorithm
merged_routes = clarke_wright_savings(distance_matrix, vehicle_capacity)

# Print the merged routes
for i, route in enumerate(merged_routes):
    print(f"Route {i+1}: {route}")


Route 1: [4, 5]
